# 05 — Portfolio Analytics (Vectorization, KRD, Credit Spread, Combined Grid)

## Objective
Demonstrate how this scales beyond single-name pricing:
- vectorized pricing across a portfolio (cashflow expansion + discount once per date)
- portfolio DV01 / convexity
- key rate duration (bucketed DV01 by curve knots)
- corporate spread pricing + spread DV01
- combined rate/spread scenario grid

This notebook depicts the following workflow:
- compute market value
- compute rate risk
- compute spread risk
- explain scenario P&L

In [1]:
import numpy as np
import pandas as pd

from fixed_income_engine.curves import bootstrap_curve_from_bills_notes
from fixed_income_engine.portfolio import (
    make_sample_portfolio,
    price_portfolio_vectorized,
    add_random_spreads,
    price_corporate_portfolio_vectorized,
)
from fixed_income_engine.risk import (
    compute_portfolio_dv01,
    compute_krd_hat,
    portfolio_spread_dv01,
)
from fixed_income_engine.scenarios import (
    run_combined_rate_spread_scenarios,
)

val_date = pd.Timestamp("2026-02-13")
settle = pd.Timestamp("2026-02-16")

market = pd.DataFrame([
    {"type": "bill", "maturity": pd.Timestamp("2026-02-20"), "quote": 0.0525, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-03-13"), "quote": 0.0520, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-05-15"), "quote": 0.0515, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-08-14"), "quote": 0.0505, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2027-02-12"), "quote": 0.0485, "day_count": "ACT/360"},
    {"type": "note", "maturity": pd.Timestamp("2028-02-15"), "quote": 0.0450, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2031-02-15"), "quote": 0.0430, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2036-02-15"), "quote": 0.0425, "coupon_freq": 2, "day_count": "30/360"},
]).sort_values("maturity").reset_index(drop=True)

curve = bootstrap_curve_from_bills_notes(market, val_date)

portfolio_df = make_sample_portfolio(n=20, val_date=val_date, seed=7)

In [2]:
priced = price_portfolio_vectorized(curve, portfolio_df, val_date, settle)
priced.head(10)

,bond_id,maturity,coupon_rate,freq,day_count,face,pv,dirty,accrued,clean,flags
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0,94.504154,94.504154,0.010606,94.493547,
1,BOND_001,2033-02-15,0.036706,2,30/360,100.0,94.369914,94.369914,0.010196,94.359718,
2,BOND_002,2033-02-15,0.035292,2,30/360,100.0,93.539742,93.539742,0.009803,93.529938,
3,BOND_003,2035-02-15,0.046705,2,30/360,100.0,100.923368,100.923368,0.012973,100.910395,
4,BOND_004,2032-02-15,0.050273,2,30/360,100.0,101.723256,101.723256,0.013965,101.709291,
5,BOND_005,2034-02-15,0.053210,2,30/360,100.0,104.870534,104.870534,0.014781,104.855754,
6,BOND_006,2035-02-15,0.079730,2,30/360,100.0,124.890127,124.890127,0.022147,124.867980,
7,BOND_007,2029-02-15,0.067560,2,30/360,100.0,104.305304,104.305304,0.018767,104.286537,
8,BOND_008,2027-02-15,0.057331,2,ACT/365,100.0,100.844864,100.844864,0.015837,100.829027,
9,BOND_009,2030-02-15,0.079338,2,30/360,100.0,110.717796,110.717796,0.022038,110.695757,


In [3]:
mv_total = priced["dirty"].sum()
dv01_total = compute_portfolio_dv01(curve, portfolio_df, val_date, settle)["dv01"].sum()

print("Total Market Value (per 100 notional sum):", mv_total)
print("Parallel DV01:", dv01_total)

Total Market Value (per 100 notional sum): 2032.5327745319332
Parallel DV01: -1.0302933377668495


In [4]:
bucket_pnl, par_dv01 = compute_krd_hat(curve, portfolio_df, val_date, settle, bp=1.0)

print("Bucket sum:", bucket_pnl.sum())
print("Parallel DV01:", par_dv01)
print("Difference:", bucket_pnl.sum() - par_dv01)

Bucket sum: -1.0303758845650464
Parallel DV01: -1.0302933377672616
Difference: -8.254679778474383e-05


In [5]:
corp_portfolio_df = add_random_spreads(portfolio_df, seed=11)
corp_priced = price_corporate_portfolio_vectorized(curve, corp_portfolio_df, val_date, settle)
corp_priced.head(10)

,bond_id,maturity,coupon_rate,freq,day_count,face,spread,pv,dirty,accrued,clean,flags
0,BOND_000,2036-02-15,0.038182,2,30/360,100.0,0.005829,90.017060,90.017060,0.010606,90.006454,
1,BOND_001,2033-02-15,0.036706,2,30/360,100.0,0.013984,86.536609,86.536609,0.010196,86.526413,
2,BOND_002,2033-02-15,0.035292,2,30/360,100.0,0.016233,84.560044,84.560044,0.009803,84.550241,
3,BOND_003,2035-02-15,0.046705,2,30/360,100.0,0.003631,98.224779,98.224779,0.012973,98.211806,
4,BOND_004,2032-02-15,0.050273,2,30/360,100.0,0.006254,98.431521,98.431521,0.013965,98.417556,
5,BOND_005,2034-02-15,0.053210,2,30/360,100.0,0.023421,89.833634,89.833634,0.014781,89.818854,
6,BOND_006,2035-02-15,0.079730,2,30/360,100.0,0.004549,121.042595,121.042595,0.022147,121.020448,
7,BOND_007,2029-02-15,0.067560,2,30/360,100.0,0.005855,102.627568,102.627568,0.018767,102.608802,
8,BOND_008,2027-02-15,0.057331,2,ACT/365,100.0,0.023863,98.506347,98.506347,0.015837,98.490510,
9,BOND_009,2030-02-15,0.079338,2,30/360,100.0,0.016681,104.392322,104.392322,0.022038,104.370284,


In [6]:
psdv01 = portfolio_spread_dv01(curve, corp_portfolio_df, val_date, settle)
print("Portfolio Spread DV01 (per 1bp):", psdv01)
print("Implied 25bp P&L from linear approx:", 25*psdv01)

Portfolio Spread DV01 (per 1bp): -0.9462587331993291
Implied 25bp P&L from linear approx: -23.656468329983227


In [7]:
combo = run_combined_rate_spread_scenarios(curve, corp_portfolio_df, val_date, settle)
pivot = combo.pivot(index="rate_shock_bp", columns="spread_shock_bp", values="total_pnl_per_100_notional")
pivot

spread_shock_bp,0,25,100
rate_shock_bp,,,
-50,48.145680,2.386735e+01,-46.533397
-25,23.867352,-2.273737e-13,-69.214847
0,0.000000,-2.346432e+01,-91.516137
25,-23.464322,-4.653340e+01,-113.444579
50,-46.533397,-6.921485e+01,-135.007333


## What we have done
- We price the portfolio by expanding cashflows, discounting each unique payment date once, and aggregating PV by bond.
- We compute DV01 by bumping the curve and repricing; we reconcile KRD buckets back to parallel DV01.
- We model spread risk by applying a flat spread to discounting: $D_{corp}=D\exp(-s\tau)$. It’s a simplified OAS intuition.
- We run a combined rate/spread grid to explain P&L and identify whether losses are rate-driven, spread-driven, or convexity residual.

## What we would upgrade in a real bank system (explicitly stated)
- business day calendars & holiday schedules
- full schedule generation from issue date with stubs and EOM rules
- multi-curve framework (OIS discounting + forward curves)
- credit curve / hazard modeling and true OAS
- curve smoothing / arbitrage-free constraints
- pricing at scale using columnar compute / distributed compute